# QR 코드 피싱 탐지기

학습된 CNN 모델을 사용하여 QR 코드 이미지가 피싱 사이트로 연결되는지 판별합니다.

In [9]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [10]:
# 모델 로드
model_path = '../best_model.h5'  # chage me
model = tf.keras.models.load_model(model_path)
print("모델 로드 완료")

모델 로드 완료


In [11]:
def preprocess_image(image_path, target_size=(128, 128)):
    """이미지를 전처리하는 함수"""
    # 이미지 로드 및 크기 조정
    img = load_img(image_path, color_mode='grayscale', target_size=target_size)
    
    # 이미지를 배열로 변환
    img_array = img_to_array(img)
    
    # 배치 차원 추가 및 정규화
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    return img_array

def predict_qr_code(image_path):
    """QR 코드 이미지의 피싱 여부를 예측하는 함수"""
    # 이미지 전처리
    processed_img = preprocess_image(image_path)
    
    # 예측 수행
    prediction = model.predict(processed_img)
    
    # 결과 해석 (0: benign, 1: phishing)
    is_phishing = prediction[0][1] > 0.5
    confidence = prediction[0][1] if is_phishing else prediction[0][0]
    
    return {
        'is_phishing': is_phishing,
        'confidence': float(confidence),
        'prediction': prediction[0]
    }

## 단일 이미지 테스트

아래 셀에서 테스트하고 싶은 QR 코드 이미지의 경로를 지정하세요.

In [12]:
# 테스트할 이미지 경로 지정
test_image_path = './badqr.png'  # chage me

In [13]:
# 예측 수행
result = predict_qr_code(test_image_path)

# 결과 출력
print(f"분석 결과:")
print(f"피싱 여부: {'피싱 의심' if result['is_phishing'] else '정상'}")
print(f"신뢰도: {result['confidence']*100:.2f}%")
print(f"\n상세 예측값:")
print(f"정상 확률: {result['prediction'][0]*100:.2f}%")
print(f"피싱 확률: {result['prediction'][1]*100:.2f}%")

1/1 [==============================] - 0s 54ms/step
분석 결과:
피싱 여부: 피싱 의심
신뢰도: 90.75%

상세 예측값:
정상 확률: 9.25%
피싱 확률: 90.75%


## 여러 이미지 일괄 테스트

특정 폴더 내의 모든 QR 코드 이미지를 테스트합니다.

In [ ]:
# 테스트할 폴더 경로 지정
test_folder = 'path/to/your/qr_codes/folder'  # chage me

In [ ]:
def batch_predict(folder_path):
    """폴더 내의 모든 이미지를 분석하는 함수"""
    results = []
    
    valid_extensions = ('.png', '.jpg', '.jpeg')
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(valid_extensions):
            image_path = os.path.join(folder_path, filename)
            try:
                result = predict_qr_code(image_path)
                results.append({
                    'filename': filename,
                    **result
                })
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    
    return results

In [ ]:
# 예측 수행
batch_results = batch_predict(test_folder)

# 결과 출력
print(f"분석된 이미지 수: {len(batch_results)}\n")
for result in batch_results:
    print(f"파일명: {result['filename']}")
    print(f"판정: {'피싱 의심' if result['is_phishing'] else '정상'}")
    print(f"신뢰도: {result['confidence']*100:.2f}%\n")